#### Draw topography map
change "time" variable to see topography at certain time

In [ ]:
import os, sys
import matplotlib
matplotlib.use('Agg')
%matplotlib inline  
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

time = 0 #change this number to draw topography at certain time

#the location and naming convention of topography files
topo_suffix = 'smooth'
topo_dir = '../data/topo_grids/'

#read in data
data = np.genfromtxt(topo_dir + '/{0}Ma_{1}.xyz'.format(time, topo_suffix))
x = data[:,0]
y = data[:,1]
z = data[:,2]

#initialize figure and basemap
plt.figure(figsize=(15, 8))
m = Basemap(llcrnrlon=-180,llcrnrlat=-90,urcrnrlon=180,urcrnrlat=90,projection='kav7', lon_0=0)
xi, yi = m(x, y)
v_min = -5000
v_max = 5000

#reorder the data so that it can be used in imshow
zz = z.reshape((181, 361))
zz = np.flipud(zz)

#intopolate data and draw map
img_data = m.transform_scalar(zz, np.arange(-180,180),np.arange(-90,90),361,181)
cs = m.imshow(img_data, alpha=0.5, vmin=v_min, vmax=v_max, cmap='terrain')

#draw meridians and parallels
m.drawparallels(np.arange(-90., 90., 10.), labels=[1,0,0,0], fontsize=10)
m.drawmeridians(np.arange(-180., 180., 45.), labels=[0,0,0,1], fontsize=10)

#draw colour bar and title
cbar = m.colorbar(cs, location='bottom', pad="10%", label='Topography (m)')
plt.title("{1} at {0} Ma".format(time, 'Topography'))

plt.show()


#### this cell may take a while to finish, be patient ...

In [ ]:
import sys
sys.path.append('../reconstruction')
sys.path.append('../utils')

from reconstruct_atom_data import *
from pyatom import Atmosphere, Hydrosphere

#create the models
atm_model = Atmosphere()
hyd_model = Hydrosphere()

#load configurations
atm_model.load_config( './config_atm.xml' )
hyd_model.load_config( './config_hyd.xml' )

#set start time, end time and time step 
start_time = 0
end_time = 6
time_step = 5

times = range(start_time, end_time+1, time_step)

#run the models
BATHYMETRY_SUFFIX = 'Ma_smooth.xyz'
for idx, time in enumerate(times):
    atm_model.run_time_slice(time)
    hyd_model.run_time_slice(time)
    if idx<len(times)-1:
        reconstruct_temperature(time,times[idx+1], BATHYMETRY_SUFFIX) 
        reconstruct_precipitation(time,times[idx+1], BATHYMETRY_SUFFIX)
        reconstruct_salinity(time,times[idx+1], BATHYMETRY_SUFFIX)



#### this cell will draw a line plot to show the temperature change over time at a given location 

In [ ]:
height = 0 
lon = 100
lat = 35

import pygplates
rotation_dir = '../reconstruction/data'
rotation_model = pygplates.RotationModel(
        rotation_dir + '/Rotations/Global_EarthByte_230-0Ma_GK07_AREPS.rot' )
static_polygon_features = pygplates.FeatureCollection(
        rotation_dir + '/ContinentalPolygons/Matthews_etal_GPC_2016_ContinentalPolygons.gpmlz' )

point_feature = pygplates.Feature()
point_feature.set_geometry(pygplates.PointOnSphere(float(lat),float(lon)))
    
assigned_point_feature = pygplates.partition_into_plates(
        static_polygon_features,
        rotation_model,
        point_feature,
        properties_to_copy = [
            pygplates.PartitionProperty.reconstruction_plate_id,
            pygplates.PartitionProperty.valid_time_period],
        )

data=[]
for time in range(start_time, end_time+1, time_step):
    pygplates.reconstruct(
            assigned_point_feature,
            rotation_model,
            reconstructed_feature_geometries,
            float(time),
            anchor_plate_id=0)
    if reconstructed_feature_geometries:
        geom =  reconstructed_feature_geometries[0].get_reconstructed_geometry()
        p = geom.to_lat_lon_list()[0]
        data.append(atm_model.get_temperature(time, height, p[1], p[0]))  
    
plt.plot(range(start_time, end_time+1, time_step), data)
plt.ylabel("Temperature (Celsius)")
plt.xlabel("Time (millon years)")
plt.show()

#### this cell draw a temperature map at a given time

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np

time = 5
height = 0

data=[]
for lat in range(-90,91):
    tmp=[]
    for lon in range(-180,181):
        tmp.append(atm_model.get_temperature(time, height, lon, lat))
    data.append(tmp)
    
plt.figure(figsize=(15, 8))

m = Basemap(llcrnrlon=-180,llcrnrlat=-90,urcrnrlon=180,urcrnrlat=90,projection='kav7', lon_0=0)

z=np.array(data)
zz = z.reshape((181, 361))

img_data = m.transform_scalar(zz, np.arange(-180,180),np.arange(-90,90),361,181)
cs = m.imshow(img_data,alpha=0.5, vmin=-60, vmax=40, cmap='jet')

m.drawparallels(np.arange(-90., 90., 10.), labels=[1,0,0,0], fontsize=10)
m.drawmeridians(np.arange(-180., 180., 45.), labels=[0,0,0,1], fontsize=10)

#draw colour bar and title
cbar = m.colorbar(cs, location='bottom', pad="10%", label='Temperature (Celsius)')
plt.title("{1} at {0} Ma".format(time, 'Temperature'))

plt.show()